In [1]:
# DATE_SPLIT = {
#     'TRAIN_START_DATE' : '2009-01-01',
#     'TRAIN_END_DATE' : '2015-09-30',
#     'VALID_START_DATE' : '2015-10-01',
#     'VALID_END_DATE' : '2015-12-31',
#     'TRADE_START_DATE' : '2016-01-01',
#     'TRADE_END_DATE' : '2020-05-08'
# }
# from get_train_valid_trade_data import get_train_valid_trade_data
# tech_list = ['open', 'high', 'low', 'close', 'volume', 'macd', 'rsi_30', 'cci_30', 'dx_30']
# # tech_list = ['macd', 'rsi_30', 'cci_30', 'dx_30']
# feature_list = ['date','tic']+tech_list
# train, valid, trade = get_train_valid_trade_data(feature_list, **DATE_SPLIT)

In [2]:
# train.to_csv('data/train.csv')
# valid.to_csv('data/valid.csv')
# trade.to_csv('data/trade.csv')

In [3]:
import pandas as pd

train = pd.read_csv('data/train.csv',index_col=0)
valid = pd.read_csv('data/valid.csv',index_col=0)
trade = pd.read_csv('data/trade.csv',index_col=0)


In [4]:
train

,date,tic,open,high,low,close,volume,macd,rsi_30,cci_30,dx_30
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.758535,746015200,-0.062362,46.725270,-32.172260,1.025271
0,2009-01-02,AMGN,58.590000,59.080002,57.750000,44.219189,6547900,0.253158,53.123358,92.999929,0.505415
0,2009-01-02,AXP,18.570000,19.520000,18.400000,15.365304,10955700,-0.780727,43.274017,-42.815877,16.568502
0,2009-01-02,BA,42.799999,45.560001,42.779999,33.941097,7010200,0.062890,51.886216,272.812751,21.032755
0,2009-01-02,CAT,44.910000,46.980000,44.709999,31.579325,7117200,0.914015,54.622472,129.598610,35.101084
...,...,...,...,...,...,...,...,...,...,...,...
1696,2015-09-29,UNH,112.889999,114.860001,111.610001,102.349861,5142700,-0.131473,45.769017,-70.056338,26.948594
1696,2015-09-29,V,67.519997,68.699997,67.199997,65.108528,11326700,-0.588576,46.072109,-128.791464,25.730401
1696,2015-09-29,VZ,43.779999,44.000000,43.340000,30.726398,15652500,-0.433851,40.343401,-124.767952,45.371808
1696,2015-09-29,WBA,82.750000,83.849998,81.449997,65.648224,4483000,-1.176599,40.770987,-154.987105,35.038123


In [5]:
tech_list = ['open', 'high', 'low', 'close', 'volume', 'macd', 'rsi_30', 'cci_30', 'dx_30']

stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(tech_list)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
29*9+29*2+1


Stock Dimension: 29, State Space: 320


320

In [6]:
stock_dim = len(train.tic.unique())
# from env.stock_portfolio_env import StockPortfolioEnv
from env.env_stocktrading import StockTradingEnv

env_kwargs = {
    "stock_dim": stock_dimension, 
    "hmax": 100, 
    "initial_amount": 1000000, 
    'num_stock_shares': [0] * stock_dimension,
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "reward_scaling": 1e-4,
    "state_space": state_space, 
    "tech_indicator_list": tech_list,
    "action_space": stock_dimension, 
    'tech_indicator_list': tech_list,
    "print_verbosity":5
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

# It will check your custom environment and output additional warnings if needed
from stable_baselines3.common.env_checker import check_env
check_env(e_train_gym)
e_train_gym = e_train_gym.get_sb_env()

In [7]:
from stable_baselines3 import A2C
from stable_baselines3 import DDPG
from stable_baselines3 import PPO

e_train_gym = StockTradingEnv(df = train, **env_kwargs).get_sb_env()
TOTAL_TIMESTEPS = 10000
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.001,
    "batch_size": 128,
}
# policy_kwargs = dict(activation_fn=torch.nn.ReLU,
#                      net_arch=dict(pi=[32,32],vf=[32,32]))
model = PPO("MlpPolicy", e_train_gym, verbose=1,**PPO_PARAMS)
model.learn(total_timesteps=TOTAL_TIMESTEPS)

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 67   |
|    iterations      | 1    |
|    time_elapsed    | 30   |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 65         |
|    iterations           | 2          |
|    time_elapsed         | 62         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.04065405 |
|    clip_fraction        | 0.404      |
|    clip_range           | 0.2        |
|    entropy_loss         | -41.1      |
|    explained_variance   | -0.0053    |
|    learning_rate        | 0.001      |
|    loss                 | 7.09       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0382    |
|    std                  | 0.998      |
|    value_loss           | 11.8       |
-----------------------------------

In [8]:
import numpy as np
import pandas as pd
from env.stock_portfolio_env import StockPortfolioEnv

# def softmax_normalization(actions):
#         numerator = np.exp(actions)
#         denominator = np.sum(numerator)
#         softmax_output = numerator/denominator
#         return softmax_output
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs).get_sb_env()

rewards=[]  # daily return rate
dones = False
obs = e_trade_gym.reset()
while not dones:
    action, _states = model.predict(obs)
    print(action)
    action = softmax_normalization(action)
    obs, reward, dones, info = e_trade_gym.step(action)
    rewards.append(reward)

rewards = np.array(rewards)
rewards = rewards+1
pd.Series(rewards.cumprod()).to_csv('results/return_rate_ppo.csv')

TypeError: StockPortfolioEnv.__init__() got an unexpected keyword argument 'hmax'